In [ ]:
MySql Replication 
=======================================

-- Replication is a copy of mysql server which is always in sync.
-- source server is called master/primary and the destination server is called slave/replica.
-- mysql replication is asyncrhonus by default.
-- mysql replication can be of all databases , some databases or even some tables.

Two methods of mysql replication 
======================================

-- traditional binary log file and position based replication.
-- GTID Global Transaction Ids based replication

Replication format
================================

-- Statement based replication.
-- Row based replication.
-- mixed based replication.


Statement based replication 
=====================================

-- binlog_format = Statement , sql statements are transferred and executed in replica.
-- consumes less space .
-- non-deterministic objects are difficult to replicate.

Row based replication
======================================

-- binlog_format = row , chnaged rows are transferred and applied in replica.
-- consumes more space .
-- cannot see what is transferred and applied in replica.

Mixed based replication 
========================================

-- binlog_format = mixed , provides best combination of data integrity and performance.
-- recommended format in most cases.

In [ ]:

Primary Server Setup 
================================


create a seperate option file replcation.cnf
=========================================================

[root@mysqlserver mysql-files]# cat replication.cnf 

[mysqld]
log-bin = /var/lib/mysql/binlog/posb_binlog
log-bin-index = /var/lib/mysql/binlog/posb_binlog.index
binlog-format = MIXED

server-id = 1
[root@mysqlserver mysql-files]# 

[root@mysqlserver mysql-files]# systemctl restart mysqld
[root@mysqlserver mysql-files]# 

mysql> show global variables like '%server_id%';
+----------------+-------+
| Variable_name  | Value |
+----------------+-------+
| server_id      | 1     |
| server_id_bits | 32    |
+----------------+-------+
2 rows in set (0.00 sec)

mysql> show global variables like '%binlog_format%';
+---------------+-------+
| Variable_name | Value |
+---------------+-------+
| binlog_format | MIXED |
+---------------+-------+
1 row in set (0.00 sec)

mysql> show global variables like '%log_bin%';
+---------------------------------+-----------------------------------------+
| Variable_name                   | Value                                   |
+---------------------------------+-----------------------------------------+
| log_bin                         | ON                                      |
| log_bin_basename                | /var/lib/mysql/binlog/posb_binlog       |
| log_bin_index                   | /var/lib/mysql/binlog/posb_binlog.index |



create a dedicated replication user 
===================================================

mysql> create user replicator identified by 'P@ssword2024';
Query OK, 0 rows affected (0.01 sec)

mysql> grant replication slave on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)

In [ ]:

Create hot backup in primary server and restore in replica server
==========================================================================

backup and prepare in primary 
-------------------------------------

[root@mysqlserver bkp_for_replication]# xtrabackup --backup --target-dir=/home/mysqluser/backups/bkp_for_replication

[root@mysqlserver backups]# xtrabackup --prepare --target-dir=/home/mysqluser/backups/bkp_for_replication


copy backup files from primary to replica server 
-----------------------------------------------------

[root@mysqlserver backups]# scp -r bkp_for_replication root@192.168.153.134:/home/mysqluser/backups/
root@192.168.153.134's password: 
ibdata1                               100%   12MB  22.4MB/s   00:00    
ibdata2                               100%   20MB  47.1MB/s   00:00    
products101.ibd                       100%   68MB  63.2MB/s   00:01    
products01.ibd                        100%   12MB  48.8MB/s   00:00    
products02.ibd                        100%   12MB  53.0MB/s   00:00    
 

stop the replica db server 
-----------------------------------------------

[root@MySqlServer2 mysql]# systemctl stop mysqld
[root@MySqlServer2 mysql]#

remove all the data , redo , undo , systemdata files in replica server
----------------------------------------------------------------------------

[root@MySqlServer2 mysql]# pwd
/var/lib/mysql
[root@MySqlServer2 mysql]# rm -rf *
[root@MySqlServer2 mysql]# 


copy all the files from the backup location to mysql datadir location
-------------------------------------------------------------------------

[root@MySqlServer2 backups]# cp -r /home/mysqluser/backups/bkp_for_replication/* /var/lib/mysql/
[root@MySqlServer2 backups]# 

[root@MySqlServer2 backups]# chown -R mysql:mysql /var/lib/mysql/
[root@MySqlServer2 backups]# 


start the mysqld server 
----------------------------

[root@MySqlServer2 backups]# systemctl start mysqld
Job for mysqld.service failed because the control process exited with error code.
See "systemctl status mysqld.service" and "journalctl -xe" for details.
[root@MySqlServer2 backups]# 

2024-05-22T16:53:11.975665Z 1 [System] [MY-013576] [InnoDB] InnoDB initialization has started.
2024-05-22T16:53:12.813406Z 1 [ERROR] [MY-012974] [InnoDB] Tablespace size stored in header is 2048 pages, 
but the sum of data file sizes is only 768 pages
2024-05-22T16:53:12.813452Z 1 [ERROR] [MY-012975] [InnoDB] Cannot start InnoDB. The tail of the system tablespace 
is missing. Have you edited innodb_data_file_path in my.cnf in an InnoDB: inappropriate way, 
removing ibdata files from there? You can set innodb_force_recovery=1 in my.cnf to force InnoDB: 
a startup if you are trying to recover a badly corrupt database.


In replica server add below in my.cnf file as primary server contains extra innodb data file 
=================================================================================================

[root@MySqlServer2 backups]# systemctl stop mysqld
[root@MySqlServer2 backups]# 

[root@MySqlServer2 backups]# cat /etc/my.cnf | grep innodb

innodb-data-file-path = ibdata1:12M;ibdata2:20M:autoextend

[root@MySqlServer2 backups]# systemctl start mysqld
[root@MySqlServer2 backups]# 
[root@MySqlServer2 backups]# systemctl status mysqld
● mysqld.service - MySQL Server
   Loaded: loaded (/usr/lib/systemd/system/mysqld.service; enabled; vendor preset: disabled)
   Active: active (running) since Wed 2024-05-22 22:36:21 IST; 11s ago


login and verify the replica server 
==========================================

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| ordersdb           |
| performance_schema |
| salesdb            |
+--------------------+
5 rows in set (0.00 sec)

In [ ]:
Replica server setup 
===========================================

create a seperate option file replcation.cnf
=========================================================

[root@MySqlServer2 ~]# cat /etc/replication.cnf 

[mysqld]
relay-log = /var/lib/mysql/relaylog/posb_relaylog
relay-log-index = /var/lib/mysql/relaylog/posb_relaylog.index

server-id = 2

skip-replica-start

read-only
[root@MySqlServer2 ~]# 

[root@MySqlServer2 mysql]# cat /etc/my.cnf | grep include

!include /etc/replication.cnf

[root@MySqlServer2 mysql]#


[root@MySqlServer2 mysql]# pwd
/var/lib/mysql
[root@MySqlServer2 mysql]# 
[root@MySqlServer2 mysql]# mkdir relaylog
[root@MySqlServer2 mysql]# 
[root@MySqlServer2 mysql]# chown -R mysql:mysql relaylog/
[root@MySqlServer2 mysql]# 


[root@MySqlServer2 ~]# systemctl restart mysqld
[root@MySqlServer2 ~]# 

mysql> show variables like '%relay%';
+---------------------------+---------------------------------------------+
| Variable_name             | Value                                       |
+---------------------------+---------------------------------------------+
| max_relay_log_size        | 0                                           |
| relay_log                 | /var/lib/mysql/relaylog/posb_relaylog       |
| relay_log_basename        | /var/lib/mysql/relaylog/posb_relaylog       |


mysql> show variables like '%server_id%';
+----------------+-------+
| Variable_name  | Value |
+----------------+-------+
| server_id      | 2     |
| server_id_bits | 32    |
+----------------+-------+
2 rows in set (0.00 sec)

In [ ]:
Replication Configuration 
============================================


Get binary log file name and position from xtrabackup 
-------------------------------------------------------------

[root@MySqlServer2 bkp_for_replication]# cat xtrabackup_binlog_info
posb_binlog.000029	157
[root@MySqlServer2 bkp_for_replication]# 


Setup replica in replica server 
-------------------------------------------

mysql> show replica status;
Empty set (0.00 sec)

mysql> change master to
    -> master_host = '192.168.153.133',
    -> master_user = 'replicator' ,
    -> master_password = 'P@ssword2024' ,
    -> master_log_file = 'posb_binlog.000029' ,
    -> master_log_pos = 157;
Query OK, 0 rows affected, 8 warnings (0.05 sec)


mysql> start slave;
Query OK, 0 rows affected, 1 warning (0.07 sec)

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Connecting to source
                  Master_Host: 192.168.153.133
                  Master_User: replicator
Last_IO_Errno: 2061
                Last_IO_Error: error connecting to master 'replicator@192.168.153.133:3306' -
                 retry-time: 60 retries: 1 message: Authentication plugin 'caching_sha2_password' 
                 reported error: Authentication requires secure connection

mysql> stop slave;
Query OK, 0 rows affected, 1 warning (0.00 sec)


restarted the replica server and is working 
========================================================

mysql> start replica;
Query OK, 0 rows affected (0.09 sec)

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: 192.168.153.133
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: posb_binlog.000030
          Read_Source_Log_Pos: 157
               Relay_Log_File: posb_relaylog.000004
                Relay_Log_Pos: 377
        Relay_Source_Log_File: posb_binlog.000030
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes

In [ ]:
Run transaction in master and check in slave whether it is replicating
==================================================================================


In primary / master 
======================================

select count(*) from products02;
'80000'

mysql> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         2 |      | 3306 |         1 | c64bbe27-185b-11ef-9f43-000c297f8903 |
+-----------+------+------+-----------+--------------------------------------+
1 row in set (0.00 sec)

mysql> show master status;
+--------------------+----------+--------------+------------------+-------------------+
| File               | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+--------------------+----------+--------------+------------------+-------------------+
| posb_binlog.000030 |      157 |              |                  |                   |
+--------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)

mysql> call ordersdb.sp_insertprod_data(80001);
Query OK, 1 row affected (10.34 sec)

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    90000 |
+----------+
1 row in set (0.14 sec)

mysql> show master status;
+--------------------+----------+--------------+------------------+-------------------+
| File               | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+--------------------+----------+--------------+------------------+-------------------+
| posb_binlog.000030 |  4350157 |              |                  |                   |
+--------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)


In replica / slave 
================================

mysql> show replica status \G
*************************** 1. row ***************************

              Source_Log_File: posb_binlog.000030
          Read_Source_Log_Pos: 4350157
               Relay_Log_File: posb_relaylog.000004
          
          Exec_Source_Log_Pos: 4350157
              Relay_Log_Space: 4350923


mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    90000 |
+----------+
1 row in set (0.20 sec)


In [ ]:
Stopping IO thread and SQL thread 
=====================================================


stop IO thread in slave 
-----------------------------------------

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Waiting for source to send event
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                  Master_Port: 3306
                Connect_Retry: 60
              Master_Log_File: posb_binlog.000030
          Read_Master_Log_Pos: 4350157
               Relay_Log_File: posb_relaylog.000004
                Relay_Log_Pos: 4350377
        Relay_Master_Log_File: posb_binlog.000030
             Slave_IO_Running: Yes
            Slave_SQL_Running: Yes

mysql> stop slave IO_THREAD;
Query OK, 0 rows affected, 1 warning (0.22 sec)

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: 
              Master_Log_File: posb_binlog.000030
          Read_Master_Log_Pos: 4350157
               Relay_Log_File: posb_relaylog.000004
                Relay_Log_Pos: 4350377
        Relay_Master_Log_File: posb_binlog.000030
             Slave_IO_Running: No
            Slave_SQL_Running: Yes


Run transaction in master 
----------------------------------------

mysql> call ordersdb.sp_insertprod_data(90001);
Query OK, 1 row affected (6.13 sec)

mysql> show master status;
+--------------------+----------+--------------+------------------+-------------------+
| File               | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+--------------------+----------+--------------+------------------+-------------------+
| posb_binlog.000030 |  8700159 |              |                  |                   |
+--------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|   100000 |
+----------+
1 row in set (0.15 sec)


Check in replica
-------------------------------------

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: 
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                  Master_Port: 3306
                Connect_Retry: 60
              Master_Log_File: posb_binlog.000030
          Read_Master_Log_Pos: 4350157
               Relay_Log_File: posb_relaylog.000004
                Relay_Log_Pos: 4350377
        Relay_Master_Log_File: posb_binlog.000030
             Slave_IO_Running: No
            Slave_SQL_Running: Yes

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|    90000 |
+----------+
1 row in set (0.15 sec)


Resume IO thread in replica and check again 
=============================================================

mysql> start slave IO_THREAD;
Query OK, 0 rows affected, 1 warning (0.20 sec)

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Waiting for source to send event
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                  Master_Port: 3306
                Connect_Retry: 60
              Master_Log_File: posb_binlog.000030
          Read_Master_Log_Pos: 8700159
               Relay_Log_File: posb_relaylog.000005
                Relay_Log_Pos: 2102248
        Relay_Master_Log_File: posb_binlog.000030
             Slave_IO_Running: Yes
            Slave_SQL_Running: Yes

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|   100000 |
+----------+
1 row in set (0.15 sec)


In [ ]:

change options in replica server and restart
===============================================================

skip-replica-start = false

log-bin = posb_replica_binlog
log-bin-index = posb_replica_binlog.index

binlog-format = MIXED


check the bin log name has changed 
-------------------------------------------------

[root@MySqlServer2 mysql]# ls -rtl posb_repl*
-rw-r-----. 1 mysql mysql 180 May 23 01:30 posb_replica_binlog.000001
-rw-r-----. 1 mysql mysql  58 May 23 01:30 posb_replica_binlog.index
-rw-r-----. 1 mysql mysql 157 May 23 01:30 posb_replica_binlog.000002
[root@MySqlServer2 mysql]# 


check whether slave is started automatically after server restart
--------------------------------------------------------------------

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Waiting for source to send event
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                  Master_Port: 3306
                Connect_Retry: 60


check the bin-log format
-----------------------------------

mysql> show variables like '%format%';
+--------------------------------------------------+---------+
| Variable_name                                    | Value   |
+--------------------------------------------------+---------+
| binlog_format                                    | MIXED   |



Now run some transactions in primary and check whether you can see sql statements in bin log 
-------------------------------------------------------------------------------------------------

mysql> show master status;
+--------------------+----------+--------------+------------------+-------------------+
| File               | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+--------------------+----------+--------------+------------------+-------------------+
| posb_binlog.000030 |  8700159 |              |                  |                   |
+--------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)

mysql> flush logs;
Query OK, 0 rows affected (0.08 sec)

mysql> show master status;
+--------------------+----------+--------------+------------------+-------------------+
| File               | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+--------------------+----------+--------------+------------------+-------------------+
| posb_binlog.000031 |      157 |              |                  |                   |
+--------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)

mysql> call ordersdb.sp_insertprod_data(100001);
Query OK, 1 row affected (10.31 sec)

mysql> select count(*) from ordersdb.products02;
+----------+
| count(*) |
+----------+
|   110000 |
+----------+
1 row in set (0.16 sec)


In primary 
=======================

[root@mysqlserver binlog]# mysqlbinlog posb_binlog.000031 > /home/mysqluser/backups/posb_binlog31.sql
[root@mysqlserver binlog]# 

[root@mysqlserver binlog]# more /home/mysqluser/backups/posb_binlog31.sql

SET TIMESTAMP=1716408509/*!*/;
insert into products02 values (  NAME_CONST('p_prod_id',100001) , concat (' product details ', NAME_CONST('p_prod_id'
,100001)) , now() )

SET TIMESTAMP=1716408509/*!*/;
insert into products02 values (  NAME_CONST('p_prod_id',100002) , concat (' product details ', NAME_CONST('p_prod_id'
,100002)) , now() )


In replica 
=========================

mysql> show master status;
+----------------------------+----------+--------------+------------------+-------------------+
| File                       | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set |
+----------------------------+----------+--------------+------------------+-------------------+
| posb_replica_binlog.000002 |  4440157 |              |                  |                   |
+----------------------------+----------+--------------+------------------+-------------------+
1 row in set (0.00 sec)

[root@MySqlServer2 mysql]# mysqlbinlog posb_replica_binlog.000002 > /home/mysqluser/backups/posbreplicabinlog2.sql
[root@MySqlServer2 mysql]# 

[root@MySqlServer2 mysql]# more /home/mysqluser/backups/posbreplicabinlog2.sql

SET TIMESTAMP=1716408509/*!*/;
insert into products02 values (  NAME_CONST('p_prod_id',100001) , concat (' product details ', NAME_CONST('p_prod_id',1000
01)) , now() )


[root@MySqlServer2 relaylog]# pwd
/var/lib/mysql/relaylog

[root@MySqlServer2 relaylog]# mysqlbinlog posb_relaylog.000011 > /home/mysqluser/backups/posbrelay11.sql
[root@MySqlServer2 relaylog]# 

[root@MySqlServer2 relaylog]# more /home/mysqluser/backups/posbrelay11.sql

SET TIMESTAMP=1716408509/*!*/;
insert into products02 values (  NAME_CONST('p_prod_id',100001) , concat (' product details ', NAME_CONST('p_prod_id',1000
01)) , now() )

In [ ]:
check tables related to master/primary and replica/slave 
========================================================================

mysql> select table_schema , table_name , table_type from information_schema.tables 
    -> where table_name like '%master%';
+--------------+-------------------+------------+
| TABLE_SCHEMA | TABLE_NAME        | TABLE_TYPE |
+--------------+-------------------+------------+
| mysql        | slave_master_info | BASE TABLE |
+--------------+-------------------+------------+
1 row in set (0.04 sec)

mysql> select table_schema , table_name , table_type from information_schema.tables  where table_name like '%slave%';
+--------------+----------------------+------------+
| TABLE_SCHEMA | TABLE_NAME           | TABLE_TYPE |
+--------------+----------------------+------------+
| mysql        | slave_relay_log_info | BASE TABLE |
| mysql        | slave_master_info    | BASE TABLE |
| mysql        | slave_worker_info    | BASE TABLE |
+--------------+----------------------+------------+
3 rows in set (0.01 sec)


mysql> select * from mysql.slave_master_info \G
*************************** 1. row ***************************
                Number_of_lines: 33
                Master_log_name: posb_binlog.000031
                 Master_log_pos: 4369799
                           Host: 192.168.153.133
                      User_name: replicator
                  User_password: P@ssword2024
                           Port: 3306
                  Connect_retry: 60
                    Enabled_ssl: 0

mysql> select * from mysql.slave_relay_log_info \G
*************************** 1. row ***************************
                             Number_of_lines: 14
                              Relay_log_name: /var/lib/mysql/relaylog/posb_relaylog.000011
                               Relay_log_pos: 4370377
                             Master_log_name: posb_binlog.000031
                              Master_log_pos: 4370157
                                   Sql_delay: 0
                           Number_of_workers: 4
                                          Id: 1
                                Channel_name: 
                   Privilege_checks_username: NULL
                   Privilege_checks_hostname: NULL
                          Require_row_format: 0
             Require_table_primary_key_check: STREAM
 Assign_gtids_to_anonymous_transactions_type: OFF
Assign_gtids_to_anonymous_transactions_value: 
1 row in set (0.00 sec)


mysql> select * from mysql.slave_worker_info \G
*************************** 1. row ***************************
                        Id: 1
            Relay_log_name: /var/lib/mysql/relaylog/posb_relaylog.000011
             Relay_log_pos: 4370377
           Master_log_name: posb_binlog.000031
            Master_log_pos: 4370157
 Checkpoint_relay_log_name: /var/lib/mysql/relaylog/posb_relaylog.000011
  Checkpoint_relay_log_pos: 4283851
Checkpoint_master_log_name: posb_binlog.000031
 Checkpoint_master_log_pos: 4283631
          Checkpoint_seqno: 197
     Checkpoint_group_size: 64
   Checkpoint_group_bitmap: 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF3
   F000000000000000000000000000000000000000000000000000000000000000000000000000000
              Channel_name: 
*************************** 2. row ***************************
                        Id: 2
            Relay_log_name: 
             Relay_log_pos: 0
           Master_log_name: 



In [ ]:

Replicate from a source database to different replica database
==========================================================================


In replica 
==============================

mysql> show variables like '%read_only%';
+-----------------------+-------+
| Variable_name         | Value |
+-----------------------+-------+
| innodb_read_only      | OFF   |
| read_only             | ON    |
| super_read_only       | OFF   |
| transaction_read_only | OFF   |
+-----------------------+-------+
4 rows in set (0.01 sec)

super user can create objects as super_read_only is OFF.

mysql> create database financedbreplica;
Query OK, 1 row affected (0.01 sec)

[root@MySqlServer2 relaylog]# systemctl stop mysqld
[root@MySqlServer2 relaylog]#


In master 
======================================

mysql> create database financedb;
Query OK, 1 row affected (0.01 sec)

mysql> create database financedb;
Query OK, 1 row affected (0.01 sec)

mysql> use financedb;
Database changed
mysql> 
mysql> create table emp_salary ( emp_id int primary key , emp_name varchar(100) , emp_salary numeric(6,2) );
Query OK, 0 rows affected (0.17 sec)

mysql> create table prod_purchase ( prod_id int primary key , prod_name varchar(100) , prod_price numeric(7,2) );
Query OK, 0 rows affected (0.03 sec)


In replica 
==========================================

[root@MySqlServer2 relaylog]# vi /etc/replication.cnf 
[root@MySqlServer2 relaylog]#

skip-replica-start

replicate-rewrite-db = financedb->financedbreplica


[root@MySqlServer2 relaylog]# systemctl start mysqld
[root@MySqlServer2 relaylog]# 

mysql> show slave status \G
*************************** 1. row ***************************
                  Slave_IO_State: 
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                  Master_Port: 3306
                  
                  Auto_Position: 0
                  Replicate_Rewrite_DB: (financedb,financedbreplica)
                  Channel_Name: 

mysql> start slave;
Query OK, 0 rows affected, 1 warning (0.28 sec)

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Waiting for source to send event
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                  Master_Port: 3306


mysql> use financedbreplica;

mysql> show tables;
+----------------------------+
| Tables_in_financedbreplica |
+----------------------------+
| emp_salary                 |
| prod_purchase              |
+----------------------------+
2 rows in set (0.00 sec)


mysql> use financedb;
Database changed
mysql> 
mysql> show tables;
Empty set (0.00 sec)


In master 
=============================

mysql> insert into emp_salary values (1001,'Ranga',200.00),(1002,'Manga',320.00),(1003,'Changa',500.00);
Query OK, 3 rows affected (0.02 sec)
Records: 3  Duplicates: 0  Warnings: 0

In replica 
===============================

mysql> select * from financedbreplica.emp_salary;
+--------+----------+------------+
| emp_id | emp_name | emp_salary |
+--------+----------+------------+
|   1001 | Ranga    |     200.00 |
|   1002 | Manga    |     320.00 |
|   1003 | Changa   |     500.00 |
+--------+----------+------------+
3 rows in set (0.00 sec)

In [ ]:
Replicate only a single database 
====================================================


In slave 
====================

[root@MySqlServer2 relaylog]# systemctl stop mysqld
[root@MySqlServer2 relaylog]# 
[root@MySqlServer2 relaylog]# vi /etc/replication.cnf 
[root@MySqlServer2 relaylog]# 

replicate-do-db = salesdb

[root@MySqlServer2 relaylog]# systemctl start mysqld
[root@MySqlServer2 relaylog]# 
[root@MySqlServer2 relaylog]# mysql

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: 
                  Master_Host: 192.168.153.133
                  Master_User: replicator
                
              Replicate_Do_DB: salesdb
              Replicate_Ignore_DB: 
              Replicate_Do_Table: 

mysql> start slave;

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Waiting for source to send event
                  Master_Host: 192.168.153.133
                  Master_User: replicator



In master 
============================

mysql> use ordersdb;

mysql> create table emp_salary ( emp_id int primary key , emp_name varchar(100) , emp_salary numeric(6,2) );
Query OK, 0 rows affected (0.14 sec)

mysql> insert into emp_salary values (1001,'Ranga',200.00),(1002,'Manga',320.00),(1003,'Changa',500.00);
Query OK, 3 rows affected (0.00 sec)
Records: 3  Duplicates: 0  Warnings: 0

mysql> select count(*) from ordersdb.emp_salary;
+----------+
| count(*) |
+----------+
|        3 |
+----------+
1 row in set (0.13 sec)


mysql> use salesdb;

mysql> create table emp_details ( emp_id int primary key , emp_name varchar(100) , emp_salary numeric(6,2) );
Query OK, 0 rows affected (0.04 sec)

mysql> insert into emp_details values (1001,'Ranga',200.00),(1002,'Manga',320.00),(1003,'Changa',500.00);
Query OK, 3 rows affected (0.24 sec)
Records: 3  Duplicates: 0  Warnings: 0

mysql> select count(*) from salesdb.emp_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+
1 row in set (0.13 sec)


In slave 
===============================


mysql> select count(*) from ordersdb.emp_salary;
ERROR 1146 (42S02): Table 'ordersdb.emp_salary' doesn't exist
mysql> 

mysql> select count(*) from salesdb.emp_details;
+----------+
| count(*) |
+----------+
|        3 |
+----------+
1 row in set (0.14 sec)


In [ ]:
Replicte 2 databases 
==============================================

replicate-do-db = salesdb
replicate-do-db = ordersdb


Replicate a single table 
==============================================

replicate-do-table = ordersdb.products01


Replicate with wild cards
=============================

replicate-wild-do-table = ordersdb.prod%
replicate-wild-do-table = order%.prod%


Replicate with ignore 
=============================

Replicate-Ignore-DB = financedb
Replicate-Ignore-Table = financedb.emp_salary
Replicate-Wild-Ignore-Table = finance%.%salary%